In [2]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

words=[]
classes=[]
doc=[]
file=open('chatbot/intents.json').read()
data=json.loads(file)

data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

In [3]:
lemmatizer=WordNetLemmatizer()
ignore=['?', '!']

for intent in data['intents']:
    for pattern in intent['patterns']:
        w=nltk.word_tokenize(pattern)
        words.extend(w)
        doc.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
        
        

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Daksh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
words=[lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore]
words=sorted(list(set(words)))

classes=sorted(list(set(classes)))
classes

['adverse_drug',
 'blood_pressure',
 'blood_pressure_search',
 'goodbye',
 'greeting',
 'hospital_search',
 'options',
 'pharmacy_search',
 'thanks']

In [6]:
training_data=[]
for d in doc:
    bag=[]
    pattern_words=d[0]
    pattern_words=[lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    output_row=list([0]*len(classes))
    output_row[classes.index(d[1])]=1
    training_data.append([bag,output_row])
    
np.random.shuffle(training_data)
training_data=np.array(training_data)
train_X=training_data[:, 0]
train_y=training_data[:, 1]

print(train_X.shape, train_y.shape)


(47,) (47,)


In [7]:
training_data.shape

(47, 2)

In [8]:
for i in range(len(train_X)):
    train_X[i]=np.asarray(train_X[i]).astype(np.float32)
    train_X[i].resize(1,88)

In [9]:
train_X[0].shape

(1, 88)

In [10]:
train_X.shape

(47,)

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Flatten

In [13]:
train_X=pad_sequences(train_X,maxlen=100)
train_y=pad_sequences(train_y,maxlen=100)

In [14]:
model=Sequential()
model.add(Flatten())
model.add(Dense(128, input_shape=train_X[0].shape, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
optim=SGD(lr=0.001)

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
type(train_y)

numpy.ndarray

In [17]:
model.fit(train_X, train_y, epochs=500, batch_size=1)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/500
47/47 [==============================] - ETA: 11s - loss: 4.6105 - acc: 0.0000e+ - ETA: 3s - loss: 4.5937 - acc: 0.0000e+00 - ETA: 1s - loss: 4.6072 - acc: 0.0000e+0 - ETA: 0s - loss: 4.6088 - acc: 0.0000e+0 - ETA: 0s - loss: 4.6022 - acc: 0.0500    - ETA: 0s - loss: 4.6040 - acc: 0.038 - ETA: 0s - loss: 4.6042 - acc: 0.031 - ETA: 0s - loss: 4.6046 - acc: 0.026 - ETA: 0s - loss: 4.6056 - acc: 0.022 - 1s 15ms/sample - loss: 4.6063 - acc: 0.0213
Epoch 2/500
47/47 [==============================] - ETA: 0s - loss: 4.5944 - acc: 0.0000e+0 - ETA: 0s - loss: 4.5876 - acc: 0.0000e+0 - ETA: 0s - loss: 4.5911 - acc: 0.0833    - ETA: 0s - loss: 4.5944 - acc: 0.055 - ETA: 0s - loss: 4.5974 - acc: 0.043 - ETA: 0s - loss: 4.5966 - acc: 0.103 - ETA: 0s - loss: 4.5996 - acc: 0.085 - ETA: 0s - loss: 4.6010 - acc: 0.075 - ETA: 0s 

47/47 [==============================] - ETA: 0s - loss: 4.4997 - acc: 0.0000e+0 - ETA: 0s - loss: 4.4545 - acc: 0.3333    - ETA: 0s - loss: 4.4459 - acc: 0.333 - ETA: 0s - loss: 4.4487 - acc: 0.277 - ETA: 0s - loss: 4.4618 - acc: 0.250 - ETA: 0s - loss: 4.4651 - acc: 0.233 - ETA: 0s - loss: 4.4677 - acc: 0.194 - ETA: 0s - loss: 4.4684 - acc: 0.190 - 0s 9ms/sample - loss: 4.4717 - acc: 0.1915
Epoch 19/500
47/47 [==============================] - ETA: 0s - loss: 4.3835 - acc: 1.000 - ETA: 0s - loss: 4.4550 - acc: 0.142 - ETA: 0s - loss: 4.4674 - acc: 0.230 - ETA: 0s - loss: 4.4843 - acc: 0.157 - ETA: 0s - loss: 4.4742 - acc: 0.160 - ETA: 0s - loss: 4.4671 - acc: 0.187 - ETA: 0s - loss: 4.4680 - acc: 0.157 - ETA: 0s - loss: 4.4689 - acc: 0.159 - 0s 8ms/sample - loss: 4.4686 - acc: 0.1702
Epoch 20/500
47/47 [==============================] - ETA: 0s - loss: 4.5345 - acc: 0.0000e+0 - ETA: 0s - loss: 4.4962 - acc: 0.0000e+0 - ETA: 0s - loss: 4.4666 - acc: 0.1538    - ETA: 0s - loss: 4.4711 

47/47 [==============================] - ETA: 0s - loss: 4.4307 - acc: 0.0000e+0 - ETA: 0s - loss: 4.3150 - acc: 0.2500    - ETA: 0s - loss: 4.2682 - acc: 0.266 - ETA: 0s - loss: 4.2679 - acc: 0.272 - ETA: 0s - loss: 4.2743 - acc: 0.259 - ETA: 0s - loss: 4.2864 - acc: 0.205 - ETA: 0s - loss: 4.2787 - acc: 0.205 - ETA: 0s - loss: 4.2731 - acc: 0.173 - 0s 9ms/sample - loss: 4.2765 - acc: 0.1702
Epoch 39/500
47/47 [==============================] - ETA: 0s - loss: 4.2423 - acc: 0.0000e+0 - ETA: 0s - loss: 4.2345 - acc: 0.0000e+0 - ETA: 0s - loss: 4.2175 - acc: 0.1538    - ETA: 0s - loss: 4.2387 - acc: 0.157 - ETA: 0s - loss: 4.2627 - acc: 0.115 - ETA: 0s - loss: 4.2570 - acc: 0.151 - ETA: 0s - loss: 4.2500 - acc: 0.200 - ETA: 0s - loss: 4.2452 - acc: 0.239 - 0s 9ms/sample - loss: 4.2474 - acc: 0.2340
Epoch 40/500
47/47 [==============================] - ETA: 0s - loss: 4.4109 - acc: 0.0000e+0 - ETA: 0s - loss: 4.2770 - acc: 0.1250    - ETA: 0s - loss: 4.2386 - acc: 0.142 - ETA: 0s - loss:

47/47 [==============================] - ETA: 0s - loss: 3.7753 - acc: 1.000 - ETA: 0s - loss: 3.9081 - acc: 0.285 - ETA: 0s - loss: 3.9813 - acc: 0.230 - ETA: 0s - loss: 3.9384 - acc: 0.315 - ETA: 0s - loss: 3.9511 - acc: 0.307 - ETA: 0s - loss: 3.9205 - acc: 0.333 - ETA: 0s - loss: 3.9350 - acc: 0.325 - ETA: 0s - loss: 3.9346 - acc: 0.282 - 0s 8ms/sample - loss: 3.9340 - acc: 0.2766
Epoch 60/500
47/47 [==============================] - ETA: 0s - loss: 3.0578 - acc: 1.000 - ETA: 0s - loss: 3.9284 - acc: 0.375 - ETA: 0s - loss: 3.9562 - acc: 0.400 - ETA: 0s - loss: 3.9454 - acc: 0.333 - ETA: 0s - loss: 3.9430 - acc: 0.250 - ETA: 0s - loss: 3.9568 - acc: 0.200 - ETA: 0s - loss: 3.9944 - acc: 0.166 - 0s 8ms/sample - loss: 3.9772 - acc: 0.1915
Epoch 61/500
47/47 [==============================] - ETA: 0s - loss: 4.2819 - acc: 0.0000e+0 - ETA: 0s - loss: 4.0099 - acc: 0.0000e+0 - ETA: 0s - loss: 3.9575 - acc: 0.0667    - ETA: 0s - loss: 3.8695 - acc: 0.227 - ETA: 0s - loss: 3.8724 - acc: 0

47/47 [==============================] - ETA: 0s - loss: 3.1233 - acc: 0.0000e+0 - ETA: 0s - loss: 3.5955 - acc: 0.0000e+0 - ETA: 0s - loss: 3.5262 - acc: 0.0667    - ETA: 0s - loss: 3.5141 - acc: 0.045 - ETA: 0s - loss: 3.4764 - acc: 0.103 - ETA: 0s - loss: 3.4543 - acc: 0.111 - ETA: 0s - loss: 3.4052 - acc: 0.116 - 0s 8ms/sample - loss: 3.3728 - acc: 0.1277
Epoch 82/500
47/47 [==============================] - ETA: 0s - loss: 2.2089 - acc: 1.000 - ETA: 0s - loss: 2.8325 - acc: 0.250 - ETA: 0s - loss: 2.9768 - acc: 0.200 - ETA: 0s - loss: 3.2307 - acc: 0.136 - ETA: 0s - loss: 3.3080 - acc: 0.172 - ETA: 0s - loss: 3.2822 - acc: 0.194 - ETA: 0s - loss: 3.2441 - acc: 0.209 - 0s 8ms/sample - loss: 3.2611 - acc: 0.1915
Epoch 83/500
47/47 [==============================] - ETA: 0s - loss: 3.6581 - acc: 0.0000e+0 - ETA: 0s - loss: 3.2290 - acc: 0.2857    - ETA: 0s - loss: 3.2812 - acc: 0.214 - ETA: 0s - loss: 3.2983 - acc: 0.142 - ETA: 0s - loss: 3.3153 - acc: 0.142 - ETA: 0s - loss: 3.2166 

...

47/47 [==============================] - ETA: 0s - loss: 0.3040 - acc: 1.000 - ETA: 0s - loss: 0.5021 - acc: 0.857 - ETA: 0s - loss: 0.6614 - acc: 0.692 - ETA: 0s - loss: 0.6267 - acc: 0.764 - ETA: 0s - loss: 0.6567 - acc: 0.782 - ETA: 0s - loss: 0.6094 - acc: 0.793 - ETA: 0s - loss: 0.6211 - acc: 0.800 - ETA: 0s - loss: 0.6615 - acc: 0.804 - 0s 9ms/sample - loss: 0.6706 - acc: 0.8085
Epoch 401/500
47/47 [==============================] - ETA: 0s - loss: 2.0444 - acc: 0.0000e+0 - ETA: 0s - loss: 0.7760 - acc: 0.8571    - ETA: 0s - loss: 0.7807 - acc: 0.769 - ETA: 0s - loss: 0.7218 - acc: 0.789 - ETA: 0s - loss: 0.6458 - acc: 0.800 - ETA: 0s - loss: 0.6158 - acc: 0.838 - ETA: 0s - loss: 0.7010 - acc: 0.810 - ETA: 0s - loss: 0.6988 - acc: 0.790 - 0s 9ms/sample - loss: 0.6738 - acc: 0.8085
Epoch 402/500
47/47 [==============================] - ETA: 0s - loss: 0.7337 - acc: 1.000 - ETA: 0s - loss: 0.6580 - acc: 0.857 - ETA: 0s - loss: 0.6089 - acc: 0.846 - ETA: 0s - loss: 0.5936 - acc: 0.8

47/47 [==============================] - ETA: 0s - loss: 0.5695 - acc: 1.000 - ETA: 0s - loss: 0.2241 - acc: 1.000 - ETA: 0s - loss: 0.4874 - acc: 0.909 - ETA: 0s - loss: 0.5219 - acc: 0.937 - ETA: 0s - loss: 0.5183 - acc: 0.954 - ETA: 0s - loss: 0.4752 - acc: 0.963 - ETA: 0s - loss: 0.4546 - acc: 0.968 - ETA: 0s - loss: 0.5532 - acc: 0.891 - ETA: 0s - loss: 0.5608 - acc: 0.860 - 0s 10ms/sample - loss: 0.6007 - acc: 0.8298
Epoch 421/500
47/47 [==============================] - ETA: 0s - loss: 1.6163 - acc: 0.0000e+0 - ETA: 0s - loss: 0.8486 - acc: 0.7143    - ETA: 0s - loss: 0.6459 - acc: 0.833 - ETA: 0s - loss: 0.6967 - acc: 0.777 - ETA: 0s - loss: 0.6148 - acc: 0.833 - ETA: 0s - loss: 0.6053 - acc: 0.800 - ETA: 0s - loss: 0.5516 - acc: 0.805 - ETA: 0s - loss: 0.5600 - acc: 0.780 - 0s 9ms/sample - loss: 0.6002 - acc: 0.7660
Epoch 422/500
47/47 [==============================] - ETA: 0s - loss: 1.1715 - acc: 0.0000e+0 - ETA: 0s - loss: 0.7591 - acc: 0.7143    - ETA: 0s - loss: 0.6316 -

47/47 [==============================] - ETA: 0s - loss: 0.0011 - acc: 1.000 - ETA: 0s - loss: 0.8936 - acc: 0.714 - ETA: 0s - loss: 0.8610 - acc: 0.769 - ETA: 0s - loss: 0.6539 - acc: 0.842 - ETA: 0s - loss: 0.5904 - acc: 0.880 - ETA: 0s - loss: 0.6006 - acc: 0.903 - ETA: 0s - loss: 0.6433 - acc: 0.837 - ETA: 0s - loss: 0.6099 - acc: 0.860 - 0s 9ms/sample - loss: 0.6151 - acc: 0.8511
Epoch 441/500
47/47 [==============================] - ETA: 0s - loss: 0.0216 - acc: 1.000 - ETA: 0s - loss: 0.4594 - acc: 0.857 - ETA: 0s - loss: 0.4560 - acc: 0.928 - ETA: 0s - loss: 0.4314 - acc: 0.950 - ETA: 0s - loss: 0.5220 - acc: 0.846 - ETA: 0s - loss: 0.5670 - acc: 0.875 - ETA: 0s - loss: 0.6085 - acc: 0.894 - ETA: 0s - loss: 0.6575 - acc: 0.886 - 0s 9ms/sample - loss: 0.6574 - acc: 0.8723
Epoch 442/500
47/47 [==============================] - ETA: 0s - loss: 0.3484 - acc: 1.000 - ETA: 0s - loss: 0.5209 - acc: 0.833 - ETA: 0s - loss: 0.6288 - acc: 0.750 - ETA: 0s - loss: 0.6919 - acc: 0.722 - ETA

47/47 [==============================] - ETA: 0s - loss: 0.3547 - acc: 1.000 - ETA: 0s - loss: 0.5579 - acc: 0.857 - ETA: 0s - loss: 0.6537 - acc: 0.769 - ETA: 0s - loss: 0.6688 - acc: 0.736 - ETA: 0s - loss: 0.5913 - acc: 0.800 - ETA: 0s - loss: 0.6191 - acc: 0.806 - ETA: 0s - loss: 0.5945 - acc: 0.810 - ETA: 0s - loss: 0.5993 - acc: 0.837 - 0s 9ms/sample - loss: 0.5880 - acc: 0.8511
Epoch 461/500
47/47 [==============================] - ETA: 0s - loss: 0.1139 - acc: 1.000 - ETA: 0s - loss: 0.6720 - acc: 0.857 - ETA: 0s - loss: 0.7697 - acc: 0.769 - ETA: 0s - loss: 0.6296 - acc: 0.789 - ETA: 0s - loss: 0.5876 - acc: 0.840 - ETA: 0s - loss: 0.6847 - acc: 0.774 - ETA: 0s - loss: 0.6822 - acc: 0.756 - ETA: 0s - loss: 0.6606 - acc: 0.790 - 0s 9ms/sample - loss: 0.6345 - acc: 0.8085
Epoch 462/500
47/47 [==============================] - ETA: 0s - loss: 0.0018 - acc: 1.000 - ETA: 0s - loss: 0.4405 - acc: 0.833 - ETA: 0s - loss: 0.3261 - acc: 0.909 - ETA: 0s - loss: 0.3947 - acc: 0.866 - ETA

47/47 [==============================] - ETA: 0s - loss: 0.6659 - acc: 1.000 - ETA: 0s - loss: 0.5710 - acc: 0.875 - ETA: 0s - loss: 0.7169 - acc: 0.866 - ETA: 0s - loss: 0.6130 - acc: 0.904 - ETA: 0s - loss: 0.6324 - acc: 0.857 - ETA: 0s - loss: 0.6387 - acc: 0.852 - ETA: 0s - loss: 0.6220 - acc: 0.850 - 0s 8ms/sample - loss: 0.6616 - acc: 0.8085
Epoch 481/500
47/47 [==============================] - ETA: 0s - loss: 0.5668 - acc: 1.000 - ETA: 0s - loss: 0.9299 - acc: 0.750 - ETA: 0s - loss: 0.7565 - acc: 0.800 - ETA: 0s - loss: 0.6462 - acc: 0.863 - ETA: 0s - loss: 0.5779 - acc: 0.896 - ETA: 0s - loss: 0.5300 - acc: 0.916 - ETA: 0s - loss: 0.5485 - acc: 0.907 - 0s 8ms/sample - loss: 0.5438 - acc: 0.9149
Epoch 482/500
47/47 [==============================] - ETA: 0s - loss: 0.0159 - acc: 1.000 - ETA: 0s - loss: 0.4918 - acc: 1.000 - ETA: 0s - loss: 0.6475 - acc: 0.933 - ETA: 0s - loss: 0.5967 - acc: 0.909 - ETA: 0s - loss: 0.4917 - acc: 0.931 - ETA: 0s - loss: 0.4999 - acc: 0.916 - ETA